<a href="https://colab.research.google.com/github/jbyrneirl/BER-Rating-AI-Model/blob/main/notebooks/data-preparation/pycaret_normalised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install --pre pycaret

In [20]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/BER_Eng_Project/Data/Run_1/BERRatingData_aa.csv", sep=";", on_bad_lines="skip", low_memory=False)
print(df.shape)

df.head()

(9997, 211)


,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,UValueFloor,...,ThirdWallAgeBandId,ThirdWallTypeId,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC
0,Co. Dublin,Semi-detached house,2019,Final,A2,49.40,118.30,0.18,0.16,0.21,...,NaN,NaN,267105014/02,0.017443,0.017443,NaN,NaN,NaN,0.330,0.345
1,Co. Dublin,Semi-detached house,2019,Final,A2,49.93,118.30,0.18,0.16,0.21,...,NaN,NaN,267105014/02,0.017443,0.017443,NaN,NaN,NaN,0.333,0.348
2,Co. Galway,Detached house,1940,Existing,D2,288.33,116.18,0.86,0.86,0.65,...,3.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Co. Wicklow,Mid-terrace house,1977,Existing,D1,230.10,85.50,0.35,0.13,0.61,...,NaN,NaN,257051043,0.000222,0.012347,NaN,NaN,NaN,NaN,NaN
4,Co. Kerry,Detached house,1939,Existing,E2,349.73,123.07,0.38,1.57,0.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Code to fix this error: LightGBMError: Do not support special JSON characters in feature name.

import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

df.head()

,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorAreasqm,UValueWall,UValueRoof,UValueFloor,...,ThirdWallAgeBandId,ThirdWallTypeId,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC
0,Co. Dublin,Semi-detached house,2019,Final,A2,49.40,118.30,0.18,0.16,0.21,...,NaN,NaN,267105014/02,0.017443,0.017443,NaN,NaN,NaN,0.330,0.345
1,Co. Dublin,Semi-detached house,2019,Final,A2,49.93,118.30,0.18,0.16,0.21,...,NaN,NaN,267105014/02,0.017443,0.017443,NaN,NaN,NaN,0.333,0.348
2,Co. Galway,Detached house,1940,Existing,D2,288.33,116.18,0.86,0.86,0.65,...,3.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Co. Wicklow,Mid-terrace house,1977,Existing,D1,230.10,85.50,0.35,0.13,0.61,...,NaN,NaN,257051043,0.000222,0.012347,NaN,NaN,NaN,NaN,NaN
4,Co. Kerry,Detached house,1939,Existing,E2,349.73,123.07,0.38,1.57,0.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
from pycaret.regression import *

# drop columns which are swaying the results

df = df.drop(['CPC', 'EPC', 'RER', 'RenewEPnren', 'RenewEPren', 'SA_Code', 'PurposeOfRating', 'EnergyRating', 'HESSchemeUpgrade', 'DateOfAssessment', 'CO2Rating', 'CO2MainSpace', 'MPCDERValue'], axis=1) # PurposeOfRating HESSchemeUpgrade DateOfAssessment CO2Rating CO2MainSpace MPCDERValue

df.head()


,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,BerRating,GroundFloorAreasqm,UValueWall,UValueRoof,UValueFloor,UValueWindow,...,SecondWallTypeId,ThirdWallType_Description,ThirdWallDescription,ThirdWallArea,ThirdWallUValue,ThirdWallIsSemiExposed,ThirdWallAgeBandId,ThirdWallTypeId,prob_smarea_error_0corr,prob_smarea_error_100corr
0,Co. Dublin,Semi-detached house,2019,Final,49.40,118.30,0.18,0.16,0.21,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017443,0.017443
1,Co. Dublin,Semi-detached house,2019,Final,49.93,118.30,0.18,0.16,0.21,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017443,0.017443
2,Co. Galway,Detached house,1940,Existing,288.33,116.18,0.86,0.86,0.65,3.1,...,5.0,Solid Mass Concrete,Semi-exposed mass concrete wal,13.81,0.99,Yes,3.0,6.0,NaN,NaN
3,Co. Wicklow,Mid-terrace house,1977,Existing,230.10,85.50,0.35,0.13,0.61,2.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000222,0.012347
4,Co. Kerry,Detached house,1939,Existing,349.73,123.07,0.38,1.57,0.76,3.1,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
s = setup(df, target="BerRating", normalize = True)

,Description,Value
0,Session id,5479
1,Target,BerRating
2,Target type,Regression
3,Original data shape,"(9997, 198)"
4,Transformed data shape,"(9997, 310)"
5,Transformed train set shape,"(6997, 310)"
6,Transformed test set shape,"(3000, 310)"
7,Ordinal features,19
8,Numeric features,149
9,Categorical features,48


In [24]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,17.3039,920.4446,28.6322,0.8532,0.1466,0.1127,9.1140
xgboost,Extreme Gradient Boosting,14.8152,974.5372,29.2102,0.8508,0.1181,0.0826,3.2820
et,Extra Trees Regressor,16.8487,1158.3077,32.2203,0.8406,0.1251,0.0874,19.9760
rf,Random Forest Regressor,18.8967,1464.1963,36.1372,0.8247,0.1410,0.0987,28.3030
br,Bayesian Ridge,23.2914,1991.5814,41.8737,0.8004,0.3385,0.1836,1.7190
ridge,Ridge Regression,23.4519,2030.1776,42.3067,0.7981,0.3399,0.1865,1.4080
lasso,Lasso Regression,23.3179,2099.2922,42.7229,0.7970,0.2870,0.1691,1.7460
llar,Lasso Least Angle Regression,23.8622,2269.1021,44.1035,0.7903,0.2784,0.1701,1.4650
huber,Huber Regressor,20.1350,2278.6338,44.2372,0.7889,0.2503,0.1384,2.8960
omp,Orthogonal Matching Pursuit,25.7246,2339.5294,45.2584,0.7837,0.3645,0.1925,1.4860


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [14]:
print(best)

LGBMRegressor(n_jobs=-1, random_state=1038)


In [25]:
finalize_model(best)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Year_of_Construction',
                                             'GroundFloorAreasqm', 'UValueWall',
                                             'UValueRoof', 'UValueFloor',
                                             'UValueWindow', 'UvalueDoor',
                                             'WallArea', 'RoofArea',
                                             'FloorArea', 'WindowArea',
                                             'DoorArea', 'NoStoreys',
                                             'HSMainSystemEfficiency',
                                             'TGDLEdition', 'HSEffAdjFactor',
                                             'HSSupplHeatFracti...
                                             'FirstWallDescription',
                                             'SecondWallDescription',
                                             'ThirdWallDescription'],
                                    transformer=TargetEncoder(cols=['CountyName',
                                                                    'FirstEnerProdComment',
                                                                    'FirstWallDescription',
                                                                    'SecondWallDescription',
                                                                    'ThirdWallDescription'],
                                                              handle_missing='return_nan'))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('actual_estimator',
                 GradientBoostingRegressor(random_state=5479))])

In [ ]:
evaluate_model(best)

In [27]:
predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,20.1615,1313.8390,36.2469,0.9373,0.1585,0.1208


,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,GroundFloorAreasqm,UValueWall,UValueRoof,UValueFloor,UValueWindow,UvalueDoor,...,ThirdWallDescription,ThirdWallArea,ThirdWallUValue,ThirdWallIsSemiExposed,ThirdWallAgeBandId,ThirdWallTypeId,prob_smarea_error_0corr,prob_smarea_error_100corr,BerRating,prediction_label
441,Dublin 7,Mid-terrace house,1950,Existing,59.599998,0.36,0.17,0.61,2.70,2.88,...,NaN,NaN,NaN,NaN,NaN,NaN,2.809867e-02,0.028099,151.240005,173.588174
2525,Co. Kerry,Detached house,1900,Existing,70.980003,2.05,2.30,1.02,4.80,3.06,...,NaN,NaN,NaN,NaN,NaN,NaN,5.072202e-03,0.050000,769.859985,716.853893
6507,Dublin 13,Mid-terrace house,1930,Existing,120.169998,2.00,2.30,0.65,2.20,2.18,...,NaN,NaN,NaN,NaN,NaN,NaN,1.868260e-02,0.018683,292.100006,331.922704
5225,Co. Mayo,Semi-detached house,2019,Final,104.339996,0.20,0.15,0.12,1.20,1.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.869999,48.068375
7952,Dublin 18,Mid-floor apartment,2010,Existing,86.900002,0.37,0.25,0.00,2.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,1.065153e-05,0.002500,99.980003,104.279642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9794,Co. Laois,Semi-detached house,1940,Existing,109.620003,0.60,1.27,0.84,3.10,3.00,...,NaN,NaN,NaN,NaN,NaN,NaN,2.927500e-08,0.002500,249.199997,239.634337
7433,Co. Kildare,Semi-detached house,1984,Existing,101.669998,0.60,0.49,0.57,2.13,3.00,...,NaN,NaN,NaN,NaN,NaN,NaN,2.218867e-04,0.012347,69.029999,123.528080
8639,Dublin 15,Semi-detached house,1981,Existing,111.129997,0.81,0.27,0.46,2.75,3.00,...,NaN,NaN,NaN,NaN,NaN,NaN,2.927500e-08,0.002500,178.960007,189.052207
826,Co. Kildare,Detached house,2006,Existing,243.000000,0.37,0.23,0.34,2.00,2.90,...,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e-02,0.050000,145.740005,131.870409


In [ ]:
# load test data
test_df = pd.read_csv("/content/drive/MyDrive/BER_Eng_Project/Data/Run_1/BERRatingData_cd.csv", sep=";", on_bad_lines="skip", low_memory=False)

# predict_model(best, test_df) # this doesn't work